In [1]:
import numpy as np
import pandas as pd
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
import os
from keras.metrics import Precision
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
df=pd.read_csv('DataFrame.csv')
dfT=df[df['Rating']==1]
dfF=df[df['Rating']==0]
texts_true=list(dfT['Review'].values)
texts_false=list(dfF['Review'].values)
texts = texts_true + texts_false
count_true = len(texts_true)
count_false = len(texts_false)
total_lines = count_true + count_false
print(count_true, count_false, total_lines)
print(texts_true)
print(texts_false)
print(texts)

2992 2987 5979
['Я только что был на ужине сегодня вечером.это было так хорошо!Спасибо', 'Отличный рецепт.Я подал это в подразделение Национальной гвардии, и им это понравилось.Многие спрашивали рецепт!', 'Это действительно потрясающий десерт !!Спасибо за еще один отличный рецепт !!', 'вкусный!Но немного мягко.Приемлемая текстура. Возможно, нужно было добавить больше шоколадных чипсов.', 'вкусный.Легко приготовить. Хороший тонкий вкус. Сохраню этот рецепт на будущее.Блюдо для развлечения.', 'Почистили куриные грудки, на вкус отлично !!!', 'Мы с мужем попробовали это в ресторане, и я умоляла девушку поделиться рецептом !!!', 'Отлично, но с гораздо меньшим количеством соли и более быстрым приготовлением, чтобы все было хрустящим.', 'Действительно легко! Нам всем очень понравилось.', 'Это была прекрасная, настоящая еда! Я использовал итальянские приправленные жестяные помидоры и сыр с ароматным черным перцем.', 'Это необходимый рецепт в моем доме. Вкус вне этого мира.', 'Очень богатый и п

In [3]:
maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»', lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts(texts)

dist = list(tokenizer.word_counts.items())
print(dist[:15])
print(texts[0][:100])

[('я', 2308), ('только', 101), ('что', 861), ('был', 519), ('на', 840), ('ужине', 5), ('сегодня', 32), ('вечером', 23), ('это', 3543), ('было', 1053), ('так', 473), ('хорошо', 667), ('спасибо', 483), ('отличный', 273), ('рецепт', 999)]
Я только что был на ужине сегодня вечером.это было так хорошо!Спасибо


In [4]:
max_text_len = 30
data = tokenizer.texts_to_sequences(texts)
data_pad = pad_sequences(data, maxlen=max_text_len)
print(data_pad)

print(list(tokenizer.word_index.items()))

X = data_pad
Y = np.array([[0, 1]]*count_true + [[1, 0]]*count_false)

[[  0   0   0 ...  21  11  20]
 [  0   0   0 ...   1  26   7]
 [  0   0   0 ... 107  31   7]
 ...
 [  0   0   0 ...   2   5 495]
 [  0   0   0 ...  10  15  18]
 [  0   0   0 ...   6 447 849]]
[('это', 1), ('не', 2), ('и', 3), ('я', 4), ('очень', 5), ('было', 6), ('рецепт', 7), ('в', 8), ('что', 9), ('на', 10), ('хорошо', 11), ('но', 12), ('с', 13), ('для', 14), ('вкус', 15), ('этот', 16), ('легко', 17), ('был', 18), ('как', 19), ('спасибо', 20), ('так', 21), ('слишком', 22), ('просто', 23), ('сделать', 24), ('вкусно', 25), ('понравилось', 26), ('мне', 27), ('действительно', 28), ('сделал', 29), ('они', 30), ('отличный', 31), ('все', 32), ('снова', 33), ('хороший', 34), ('много', 35), ('меня', 36), ('были', 37), ('у', 38), ('чтобы', 39), ('мы', 40), ('их', 41), ('за', 42), ('его', 43), ('из', 44), ('бы', 45), ('то', 46), ('использовал', 47), ('вкуса', 48), ('извините', 49), ('когда', 50), ('больше', 51), ('мой', 52), ('вы', 53), ('соус', 54), ('добавил', 55), ('немного', 56), ('даже', 5

In [5]:
indeces = np.random.choice(X.shape[0], size=X.shape[0], replace=False)
X = X[indeces]
Y = Y[indeces]

X_train=X[:4801]
Y_train=Y[:4801]
X_test=X[4801:6001]
Y_test=Y[4801:6001]

model = Sequential()
model.add(Embedding(maxWordsCount, 128, input_length=max_text_len))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(64))
model.add(Dense(2, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy', Precision()], optimizer=Adam(0.001))

history = model.fit(X_train, Y_train, batch_size=16, epochs=25, validation_data = (X_test, Y_test))

reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))
#0.0001, 32, 50


def sequence_to_text(list_of_indices):
    words = [reverse_word_map.get(letter) for letter in list_of_indices]
    return (words)



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 30, 128)           128000    
                                                                 
 lstm (LSTM)                 (None, 30, 128)           131584    
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 2)                 130       
                                                                 
Total params: 309122 (1.18 MB)
Trainable params: 309122 (1.18 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/25


301/301 [==============================] - 15s 32ms/step - loss: 0.3395 - accuracy: 0.8504 - precision: 0.8504 - val_loss: 0.2397 - val_ac

In [6]:
t = "Просто вкусно, этого достаточно, спасибо за рецепт!".lower()
##Тонкое сочетания ингридиентов сделало рецепт очень вкусным, всем понравилось! Приготовлю это ещё
##Просто вкусно, этого достаточно, спасибо за рецепт!
data = tokenizer.texts_to_sequences([t])
data_pad = pad_sequences(data, maxlen=max_text_len)
print(sequence_to_text(data[0]))

res = model.predict(data_pad)
print(res, np.argmax(res))

if np.argmax(res):
    print('Positive')
else:
    print('Negative')

print("Test-Accuracy:", np.mean(history.history["val_accuracy"]))

# Оценка модели на тестовом наборе
loss, accuracy, precision = model.evaluate(X_test, Y_test)

# Печать значения precision
print('Precision:', precision)
print("Accuracy:", accuracy)

['просто', 'вкусно', 'этого', 'достаточно', 'спасибо', 'за', 'рецепт']
1/1 [==============================] - 1s 952ms/step
[[5.1759826e-06 9.9999487e-01]] 1
Positive
Test-Accuracy: 0.9029881167411804
37/37 [==============================] - 1s 15ms/step - loss: 0.7151 - accuracy: 0.9024 - precision: 0.9024
Precision: 0.9023768901824951
Accuracy: 0.9023768901824951


In [7]:
from sklearn.metrics import confusion_matrix


y_pred_probs = model.predict(X_test)  # Предсказания вероятностей
y_pred = np.argmax(y_pred_probs, axis=1)  # Преобразование вероятностей в метки классов

# Истинные метки
y_true = np.argmax(Y_test, axis=1)  # Преобразование one-hot энкодированных меток в метки классов

# Вычисление матрицы ошибок
cm = confusion_matrix(y_true, y_pred)

# Извлечение значений TP, TN, FP и FN из матрицы ошибок
# Для бинарной классификации (2 класса)
# cm = [[TN, FP],
#       [FN, TP]]

TN, FP, FN, TP = cm.ravel()

print(f'True Positives (TP): {TP}')
print(f'True Negatives (TN): {TN}')
print(f'False Positives (FP): {FP}')
print(f'False Negatives (FN): {FN}')

37/37 [==============================] - 1s 17ms/step
True Positives (TP): 525
True Negatives (TN): 538
False Positives (FP): 54
False Negatives (FN): 61


In [8]:
# import io
# import json

# model.save('sentiment.h5')

# tokenizer_json = tokenizer.to_json()
# with io.open('tokenizer.json', 'w', encoding='utf-8') as f:
#     f.write(json.dumps(tokenizer_json, ensure_ascii=False))